In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.33.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


In [ ]:
from typing import Dict, List, Any
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else torch.float16

class EndpointHandler:
    def __init__(self, path=""):

        tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            path,
            return_dict=True,
            device_map="auto",
            load_in_8bit=True,
            torch_dtype=dtype,
            trust_remote_code=True,
        )

        generation_config = model.generation_config
        generation_config.max_new_tokens = 60
        generation_config.temperature = 0
        generation_config.num_return_sequences = 1
        generation_config.pad_token_id = tokenizer.eos_token_id
        generation_config.eos_token_id = tokenizer.eos_token_id
        self.generation_config = generation_config

        self.pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer
        )

    def __call__(self, data: Dict[str, Any]) -> Dict[str, Any]:
        prompt = data.pop("inputs", data)
        result = self.pipeline(prompt, generation_config=self.generation_config)
        return result

In [ ]:
model_id = "Hawk28/CodeLlama-13B-RP"
my_handler = EndpointHandler(model_id)

(…)3B-RP/resolve/main/tokenizer_config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

(…)Llama-13B-RP/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-RP/resolve/main/special_tokens_map.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

(…)odeLlama-13B-RP/resolve/main/config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

In [ ]:
prompt = """### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
"""

In [ ]:
prediction = my_handler({"inputs": prompt})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(prediction)

[{'generated_text': '### Complete SQL query only and with no explanation\n### SQL tables followed by foreign key information:\n#\n# Books ( book_id, title, author_id, genre, publication_year )\n# Authors ( author_id, author_name, birth_year )\n# Members ( member_id, member_name, join_date )\n# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )\n#\n# BorrowedBooks.book_id can be joined with Books.book_id\n# BorrowedBooks.member_id can be joined with Members.member_id\n# Authors.author_id can be joined with Books.author_id\n#\n### Question:\n#\n# What are the titles and genres of all the books?\n#\n### SQL:\n'}]


In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Hawk28/Llama-7B-RP")
model = AutoModelForCausalLM.from_pretrained("Hawk28/Llama-7B-RP", load_in_8bit=True)

(…)7B-RP/resolve/main/tokenizer_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

(…)/Llama-7B-RP/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-RP/resolve/main/special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

(…)k28/Llama-7B-RP/resolve/main/config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    attention_mask = torch.ones(input_ids.shape, dtype=torch.bool).to("cuda")
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100, attention_mask=attention_mask)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
#
# SELECT title ,  genre FROM books
#
### End.



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the number of books published in each genre
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the number of books published in each genre
#
### SQL:
#
# SELECT DISTINCT T1.publication_year ,  T2.genre FROM books AS T1 JOIN authors AS T2 ON T1.author_id  =  T2.author_id GROUP BY T2.genre
#
### End.
###
### SQL:
#
# SELECT DISTINCT T1.publication_year ,  T2.genre FROM books AS T1 JOIN authors AS T


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the most borrowed book
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the most borrowed book
#
### SQL:
#
# SELECT book_id FROM borrowedbooks ORDER BY count(*) DESC LIMIT 1
#
### End.



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the IDs, titles, and publication years of "Fantasy" genre books?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the IDs, titles, and publication years of "Fantasy" genre books?
#
### SQL:
#
# SELECT book_id ,  title ,  publication_year FROM books WHERE genre  =  "Fantasy"
#
### End.



In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of book titles and their respective authors?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of book titles and their respective authors?
#
### SQL:
#
# SELECT title ,  author_name FROM books
#
### End.
###
### SQL:
#
# SELECT title ,  author_name FROM books
#
### End.
###
### SQL:
#
# SELECT title ,  author_name FROM books
#
### End.
###
### SQL:
#
# SELECT title ,  author_name FROM books
#
##


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Which members borrowed books along with the borrow and return dates?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Which members borrowed books along with the borrow and return dates?
#
### SQL:
#
# SELECT member_id ,  borrow_date ,  return_date FROM borrowedbooks
#
### End.
###
### SQL:
#
# SELECT member_id ,  borrow_date ,  return_date FROM borrowedbooks
#
### End.
###
### SQL:
#
# SELECT member_id ,  borrow_date ,  return_date FROM borrowedbooks
#
###


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and authors of books by authors born after 1950?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and authors of books by authors born after 1950?
#
### SQL:
#
# SELECT title ,  author_name FROM books JOIN authors ON authors.author_id  =  books.author_id WHERE authors.birth_year  >  1950
#
### End.
###
### SQL:
#
# SELECT title ,  author_name FROM books JOIN authors ON authors.author_id  =  books.author_id WHERE authors.birth_year  >  1950


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Could you provide a list of unique genres available in the collection?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Could you provide a list of unique genres available in the collection?
#
### SQL:
#
# SELECT DISTINCT genre FROM books
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Show the members who joined before 1st January 2022 (Format: DD-MM-YYYY) and the books they borrowed:
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Show the members who joined before 1st January 2022 (Format: DD-MM-YYYY) and the books they borrowed:
#
### SQL:
#
# SELECT DISTINCT member_name ,  borrow_date FROM borrowedbooks WHERE borrow_date  <  '01-01-2022' AND member_id IN ( SELECT member_id FROM members WHERE join_date  <  '01-01-2022' )
#
### End.
#
### SQL:
#
# SELECT DISTINCT member_name ,  borrow_date


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of members along with the number of books they have borrowed, sorted by the number of borrows in descending order?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of members along with the number of books they have borrowed, sorted by the number of borrows in descending order?
#
### SQL:
#
# SELECT count(*) ,  member_name FROM borrowedbooks GROUP BY member_id ORDER BY count(*) DESC
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
### End.
#
###


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What is the earliest publication year of a book in the "Fiction" genre?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    attention_mask = torch.ones(input_ids.shape, dtype=torch.bool).to("cuda")
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100, attention_mask=attention_mask)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What is the earliest publication year of a book in the "Fiction" genre?
#
### SQL:
#
# SELECT MIN(publication_year) FROM books WHERE genre  =  "Fiction"
#
### End.
#
### SQL:
#
# SELECT MIN(publication_year) FROM books WHERE genre  =  "Fiction"
#
### End.
#
### SQL:
#
# SELECT MIN(publication_year) FROM books WHERE genre  =  "Fiction"

